In [1]:
import torch
import random
import numpy as np

from datasets import load_dataset
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from transformers import PretrainedConfig, Trainer, TrainingArguments

from modules import TransformerClassifier, MLP, MoE, MoDE


def seed_all(seed: int = 0) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def get_gpu_device() -> torch.device:
    if torch.cuda.is_available():
        device_str = "cuda"     # GPU
    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        device_str = "mps"      # Apple silicon
    else:
        print("Warning: No GPU found, using CPU instead.")
        device_str = "cpu"      # CPU
    return torch.device(device_str)

seed_all()

## Configs

In [2]:
base_config = dict(
    vocab_size=5000,
    max_position_embeddings=256,
    num_attention_heads=8,
    num_hidden_layers=4,
    hidden_dropout_prob=0.1,
    hidden_size=128,
    intermediate_size=512,
    num_labels=2,
    bias=True
)

standard_config = PretrainedConfig(
    **base_config,
    ff_cls=MLP,
    mh_moe=False
)

moe_config = PretrainedConfig(
    **base_config,
    num_experts=4,
    capacity_factor=2.0,
    num_experts_per_token=1,
    ff_cls=MoE,
    mh_moe=False
)

mode_config = PretrainedConfig(
    **base_config,
    num_experts=5, # add 1 no-op expert
    capacity_factor=2.0,
    num_experts_per_token=1,
    ff_cls=MoDE,
    mh_moe=False
)

mh_moe_config = PretrainedConfig(
    **base_config,
    num_experts=4,
    capacity_factor=2.0,
    num_experts_per_token=1,
    ff_cls=MoE,
    mh_moe=True
)

mh_mode_config = PretrainedConfig(
    **base_config,
    num_experts=5, # add 1 no-op expert
    capacity_factor=2.0,
    num_experts_per_token=1,
    ff_cls=MoDE,
    mh_moe=True
)

## Tokenizer Training

In [3]:
dataset = load_dataset('imdb')

tokenizer = ByteLevelBPETokenizer()
tokenizer.train_from_iterator(
    dataset['train']['text'],
    vocab_size=base_config['vocab_size'],
    special_tokens=["<s>", "</s>", "<pad>"],
    min_frequency=2
)
tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)

tokenizer.enable_truncation(max_length=base_config['max_position_embeddings'])
tokenizer.enable_padding(pad_id=tokenizer.token_to_id("<pad>"), pad_token="<pad>", length=base_config['max_position_embeddings'])
tokenizer.model_max_length = base_config['max_position_embeddings']
tokenizer.pad_token = "<pad>"


def tokenize(row):
    return {
        'input_ids': tokenizer.encode(row['text']).ids,
    }

tokenized_dataset = dataset.map(tokenize)

## Train Vanilla Transformer

In [5]:
training_args = TrainingArguments(
    output_dir='./results_vanilla',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.001,
    logging_dir='./logs',
)

model = TransformerClassifier(standard_config)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
)

trainer.train()
trainer.evaluate()

  0%|          | 0/9375 [00:00<?, ?it/s]

{'loss': 0.7554, 'grad_norm': 11.673501014709473, 'learning_rate': 5e-05, 'epoch': 0.16}
{'loss': 0.7101, 'grad_norm': 9.985132217407227, 'learning_rate': 4.71830985915493e-05, 'epoch': 0.32}
{'loss': 0.6871, 'grad_norm': 3.6002533435821533, 'learning_rate': 4.436619718309859e-05, 'epoch': 0.48}
{'loss': 0.6654, 'grad_norm': 4.413424015045166, 'learning_rate': 4.154929577464789e-05, 'epoch': 0.64}
{'loss': 0.6496, 'grad_norm': 7.207448482513428, 'learning_rate': 3.8732394366197184e-05, 'epoch': 0.8}
{'loss': 0.643, 'grad_norm': 5.3875555992126465, 'learning_rate': 3.5915492957746486e-05, 'epoch': 0.96}
{'loss': 0.6326, 'grad_norm': 10.878851890563965, 'learning_rate': 3.3098591549295775e-05, 'epoch': 1.12}
{'loss': 0.6077, 'grad_norm': 6.362067222595215, 'learning_rate': 3.028169014084507e-05, 'epoch': 1.28}
{'loss': 0.5738, 'grad_norm': 5.495521068572998, 'learning_rate': 2.746478873239437e-05, 'epoch': 1.44}
{'loss': 0.591, 'grad_norm': 2.6917409896850586, 'learning_rate': 2.46478873

  0%|          | 0/3125 [00:00<?, ?it/s]

{'eval_loss': 0.5624286532402039,
 'eval_runtime': 53.9324,
 'eval_samples_per_second': 463.543,
 'eval_steps_per_second': 57.943,
 'epoch': 3.0}

## Train MoE

In [4]:
training_args = TrainingArguments(
    output_dir='./results_moe',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.001,
    logging_dir='./logs',
)

model_moe = TransformerClassifier(moe_config)
trainer = Trainer(
    model=model_moe,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
)

trainer.train()
trainer.evaluate()

  0%|          | 0/9375 [00:00<?, ?it/s]

{'loss': 0.8296, 'grad_norm': 5.203965663909912, 'learning_rate': 5e-05, 'epoch': 0.16}
{'loss': 0.6789, 'grad_norm': 10.183759689331055, 'learning_rate': 4.71830985915493e-05, 'epoch': 0.32}
{'loss': 0.6717, 'grad_norm': 4.524587631225586, 'learning_rate': 4.436619718309859e-05, 'epoch': 0.48}
{'loss': 0.653, 'grad_norm': 6.4882378578186035, 'learning_rate': 4.154929577464789e-05, 'epoch': 0.64}
{'loss': 0.6514, 'grad_norm': 7.845848083496094, 'learning_rate': 3.8732394366197184e-05, 'epoch': 0.8}
{'loss': 0.6504, 'grad_norm': 6.5309247970581055, 'learning_rate': 3.5915492957746486e-05, 'epoch': 0.96}
{'loss': 0.6398, 'grad_norm': 6.760333061218262, 'learning_rate': 3.3098591549295775e-05, 'epoch': 1.12}
{'loss': 0.6171, 'grad_norm': 8.21616268157959, 'learning_rate': 3.028169014084507e-05, 'epoch': 1.28}
{'loss': 0.5931, 'grad_norm': 5.277225494384766, 'learning_rate': 2.746478873239437e-05, 'epoch': 1.44}
{'loss': 0.6122, 'grad_norm': 5.520471096038818, 'learning_rate': 2.4647887323

  0%|          | 0/3125 [00:00<?, ?it/s]

{'eval_loss': 0.5792981386184692,
 'eval_runtime': 72.2348,
 'eval_samples_per_second': 346.094,
 'eval_steps_per_second': 43.262,
 'epoch': 3.0}

## Train MoDE

In [4]:
training_args = TrainingArguments(
    output_dir='./results_mode',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.001,
    logging_dir='./logs',
)

model_mode = TransformerClassifier(mode_config)
trainer = Trainer(
    model=model_mode,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
)

trainer.train()
trainer.evaluate()

  0%|          | 0/9375 [00:00<?, ?it/s]

{'loss': 0.8452, 'grad_norm': 4.549673080444336, 'learning_rate': 5e-05, 'epoch': 0.16}
{'loss': 0.6961, 'grad_norm': 12.047969818115234, 'learning_rate': 4.71830985915493e-05, 'epoch': 0.32}
{'loss': 0.6796, 'grad_norm': 8.10474681854248, 'learning_rate': 4.436619718309859e-05, 'epoch': 0.48}
{'loss': 0.6528, 'grad_norm': 7.605819225311279, 'learning_rate': 4.154929577464789e-05, 'epoch': 0.64}
{'loss': 0.6544, 'grad_norm': 9.052936553955078, 'learning_rate': 3.8732394366197184e-05, 'epoch': 0.8}
{'loss': 0.6494, 'grad_norm': 8.182595252990723, 'learning_rate': 3.5915492957746486e-05, 'epoch': 0.96}
{'loss': 0.6406, 'grad_norm': 9.334197044372559, 'learning_rate': 3.3098591549295775e-05, 'epoch': 1.12}
{'loss': 0.6223, 'grad_norm': 8.65141487121582, 'learning_rate': 3.028169014084507e-05, 'epoch': 1.28}
{'loss': 0.598, 'grad_norm': 5.2792277336120605, 'learning_rate': 2.746478873239437e-05, 'epoch': 1.44}
{'loss': 0.6172, 'grad_norm': 5.572335720062256, 'learning_rate': 2.464788732394

  0%|          | 0/3125 [00:00<?, ?it/s]

{'eval_loss': 0.582165539264679,
 'eval_runtime': 74.8182,
 'eval_samples_per_second': 334.143,
 'eval_steps_per_second': 41.768,
 'epoch': 3.0}

## Train MH-MoE

In [4]:
training_args = TrainingArguments(
    output_dir='./results_mh-moe',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.001,
    logging_dir='./logs',
)

model_mh_moe = TransformerClassifier(mh_moe_config)
trainer = Trainer(
    model=model_mh_moe,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
)

trainer.train()
trainer.evaluate()

  0%|          | 0/9375 [00:00<?, ?it/s]

{'loss': 0.7514, 'grad_norm': 4.770580291748047, 'learning_rate': 5e-05, 'epoch': 0.16}
{'loss': 0.6852, 'grad_norm': 11.751579284667969, 'learning_rate': 4.71830985915493e-05, 'epoch': 0.32}
{'loss': 0.679, 'grad_norm': 6.661434173583984, 'learning_rate': 4.436619718309859e-05, 'epoch': 0.48}
{'loss': 0.6546, 'grad_norm': 6.11175537109375, 'learning_rate': 4.154929577464789e-05, 'epoch': 0.64}
{'loss': 0.659, 'grad_norm': 7.312709808349609, 'learning_rate': 3.8732394366197184e-05, 'epoch': 0.8}
{'loss': 0.6487, 'grad_norm': 6.433994770050049, 'learning_rate': 3.5915492957746486e-05, 'epoch': 0.96}
{'loss': 0.6401, 'grad_norm': 10.707786560058594, 'learning_rate': 3.3098591549295775e-05, 'epoch': 1.12}
{'loss': 0.6253, 'grad_norm': 7.640051364898682, 'learning_rate': 3.028169014084507e-05, 'epoch': 1.28}
{'loss': 0.596, 'grad_norm': 5.8375067710876465, 'learning_rate': 2.746478873239437e-05, 'epoch': 1.44}
{'loss': 0.6098, 'grad_norm': 6.63223123550415, 'learning_rate': 2.4647887323943

  0%|          | 0/3125 [00:00<?, ?it/s]

{'eval_loss': 0.5796149373054504,
 'eval_runtime': 156.5182,
 'eval_samples_per_second': 159.726,
 'eval_steps_per_second': 19.966,
 'epoch': 3.0}

## Train MH-MoDE

In [4]:
training_args = TrainingArguments(
    output_dir='./results_mh-mode',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.001,
    logging_dir='./logs',
)

model_mh_mode = TransformerClassifier(mh_mode_config)
trainer = Trainer(
    model=model_mh_mode,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
)

trainer.train()
trainer.evaluate()

  0%|          | 0/9375 [00:00<?, ?it/s]

{'loss': 0.7399, 'grad_norm': 4.209444522857666, 'learning_rate': 5e-05, 'epoch': 0.16}
{'loss': 0.6829, 'grad_norm': 10.695562362670898, 'learning_rate': 4.71830985915493e-05, 'epoch': 0.32}
{'loss': 0.6797, 'grad_norm': 5.148598670959473, 'learning_rate': 4.436619718309859e-05, 'epoch': 0.48}
{'loss': 0.6552, 'grad_norm': 5.054625511169434, 'learning_rate': 4.154929577464789e-05, 'epoch': 0.64}
{'loss': 0.6515, 'grad_norm': 6.36778450012207, 'learning_rate': 3.8732394366197184e-05, 'epoch': 0.8}
{'loss': 0.6522, 'grad_norm': 6.712286472320557, 'learning_rate': 3.5915492957746486e-05, 'epoch': 0.96}
{'loss': 0.641, 'grad_norm': 8.144624710083008, 'learning_rate': 3.3098591549295775e-05, 'epoch': 1.12}
{'loss': 0.6236, 'grad_norm': 7.289916038513184, 'learning_rate': 3.028169014084507e-05, 'epoch': 1.28}
{'loss': 0.5931, 'grad_norm': 4.87639856338501, 'learning_rate': 2.746478873239437e-05, 'epoch': 1.44}
{'loss': 0.6118, 'grad_norm': 3.6915459632873535, 'learning_rate': 2.464788732394

  0%|          | 0/3125 [00:00<?, ?it/s]

{'eval_loss': 0.5823717713356018,
 'eval_runtime': 149.6315,
 'eval_samples_per_second': 167.077,
 'eval_steps_per_second': 20.885,
 'epoch': 3.0}